In [3]:
# ============================================================
# RomKorr MapBuilder – Variante A (robust, CSV-only)
# Erzeugt: romkorr_map_fixed.html
#
# Fixes:
# - Map-Referenz im JS korrekt: window[map_name] (sonst laufen Filter/Linien/Info nicht)
# - Heatmap in benannter FeatureGroup ("Heatmap") statt "macro_element_..."
# - Nur 1 Marker pro Koordinate (Aggregation dispatch+destination)
# - "Alle Korrespondenzen" öffnet Sidebar mit Paging + CSV-Download
# - Linien werden clientseitig aus JSON neu gezeichnet + filterbar
# ============================================================

from __future__ import annotations

from pathlib import Path
import json

import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
from branca.element import MacroElement, Template, Element


# -----------------------------
# Pfade / Output
# -----------------------------
LETTERS_CSV = Path("letters_master.csv")  # ggf. anpassen
OUT_HTML = Path("romkorr_map_fixed.html")

CENTER = (51.0, 10.0)
ZOOM_START = 6

TOP_N_PERSONS = 80
EXAMPLES_PER_PLACE = 6
PAGE_SIZE_PLACE_LIST = 50
COORD_ROUND = 5


# -----------------------------
# Helper
# -----------------------------
def to_de_date(date_iso: str) -> str:
    """YYYY-MM-DD -> DD.MM.YYYY (falls parsebar), sonst Original."""
    if not date_iso or not isinstance(date_iso, str):
        return ""
    parts = date_iso.split("-")
    if len(parts) >= 3 and all(p.isdigit() for p in parts[:3]):
        y, m, d = parts[:3]
        if len(y) == 4:
            return f"{d.zfill(2)}.{m.zfill(2)}.{y}"
    return date_iso


def coord_key(lat: float, lon: float, ndigits: int = COORD_ROUND) -> str:
    return f"{round(float(lat), ndigits):.{ndigits}f},{round(float(lon), ndigits):.{ndigits}f}"


# -----------------------------
# Daten laden
# -----------------------------
letters = pd.read_csv(LETTERS_CSV)

# leichte Normalisierung für Sender/Recipient falls anders benannt
rename_map = {}
for c in letters.columns:
    if c.lower() == "sender":
        rename_map[c] = "Sender"
    if c.lower() == "recipient":
        rename_map[c] = "Recipient"
letters = letters.rename(columns=rename_map)

# date_iso sicher als string
if "date_iso" not in letters.columns:
    letters["date_iso"] = ""
letters["date_iso"] = letters["date_iso"].fillna("").astype(str)
letters["date_de"] = letters["date_iso"].map(to_de_date)

# Pflichtspalten prüfen
need_cols = {
    "dispatch_lat", "dispatch_lon",
    "destination_lat", "destination_lon",
    "Sender", "Recipient",
    "link_canonical", "date_iso",
}
missing = [c for c in sorted(need_cols) if c not in letters.columns]
if missing:
    raise RuntimeError(f"Missing columns in letters_master.csv: {missing}")


# -----------------------------
# Top Personen (Filter)
# -----------------------------
persons = pd.concat(
    [letters["Sender"].dropna().astype(str), letters["Recipient"].dropna().astype(str)],
    ignore_index=True,
)
persons = persons[persons.str.len() > 0]
top_persons = persons.value_counts().head(TOP_N_PERSONS).index.tolist()


# -----------------------------
# Ort -> Briefe Index (für Sidebar)
#   Wir indexieren sowohl dispatch als auch destination Orte
# -----------------------------
place_index: dict[str, list[dict]] = {}

def add_place_entries(lat_col: str, lon_col: str) -> None:
    sub = letters.dropna(subset=[lat_col, lon_col]).copy()
    sub["k"] = [coord_key(a, b) for a, b in zip(sub[lat_col], sub[lon_col])]
    sub = sub.sort_values(["date_iso"], na_position="last")

    for _, r in sub.iterrows():
        k = r["k"]
        place_index.setdefault(k, []).append({
            "s": str(r.get("Sender", "?")),
            "r": str(r.get("Recipient", "?")),
            "d": str(r.get("date_de", "")),
            "iso": str(r.get("date_iso", "")) if pd.notnull(r.get("date_iso")) else "",
            "u": str(r.get("link_canonical", "#")),
            "id": int(r.get("letter_id")) if ("letter_id" in r and pd.notnull(r.get("letter_id"))) else None,
        })

add_place_entries("dispatch_lat", "dispatch_lon")
add_place_entries("destination_lat", "destination_lon")


# -----------------------------
# Orte aggregieren (Hotspots/Heatmap)
#   -> eine Zeile pro gerundetem Koordinaten-Key
#   -> w = Anzahl Briefe, die diesen Ort berühren
# -----------------------------
def build_place_agg() -> pd.DataFrame:
    rows = []

    for lat_col, lon_col, name_col in [
        ("dispatch_lat", "dispatch_lon", "place_dispatch"),
        ("destination_lat", "destination_lon", "place_destination"),
    ]:
        tmp = letters.dropna(subset=[lat_col, lon_col]).copy()
        tmp["k"] = [coord_key(a, b) for a, b in zip(tmp[lat_col], tmp[lon_col])]
        tmp["lat"] = tmp[lat_col].astype(float)
        tmp["lon"] = tmp[lon_col].astype(float)
        tmp["place"] = tmp[name_col].astype(str) if name_col in tmp.columns else "Ort (unbekannt)"
        rows.append(tmp[["k", "lat", "lon", "place"]])

    if not rows:
        return pd.DataFrame(columns=["k", "lat", "lon", "w", "place"])

    allp = pd.concat(rows, ignore_index=True)

    def mode_place(s: pd.Series) -> str:
        s = s.dropna()
        if s.empty:
            return "Ort (unbekannt)"
        m = s.mode()
        return str(m.iloc[0]) if not m.empty else str(s.iloc[0])

    agg = allp.groupby("k", as_index=False).agg(
        lat=("lat", "mean"),
        lon=("lon", "mean"),
        w=("k", "size"),
        place=("place", mode_place),
    )
    return agg

agg = build_place_agg()
place_name_by_key = dict(zip(agg["k"], agg["place"]))


# Payload für Sidebar: Ortname + Items
place_payload = {k: {"place": place_name_by_key.get(k, "Ort (unbekannt)"), "items": v}
                 for k, v in place_index.items()}


# -----------------------------
# Linien-Payload (alle Verbindungen)
# -----------------------------
lines_df = letters.dropna(subset=["dispatch_lat", "dispatch_lon", "destination_lat", "destination_lon"]).copy()

lines_payload = []
for r in lines_df.itertuples(index=False):
    iso = getattr(r, "date_iso", "")
    iso = "" if iso is None else str(iso)
    lines_payload.append({
        "a": [float(getattr(r, "dispatch_lat")), float(getattr(r, "dispatch_lon"))],
        "b": [float(getattr(r, "destination_lat")), float(getattr(r, "destination_lon"))],
        "s": str(getattr(r, "Sender")),
        "r": str(getattr(r, "Recipient")),
        "d": to_de_date(iso),
        "iso": iso,
        "km": float(getattr(r, "distance_km")) if hasattr(r, "distance_km") and pd.notnull(getattr(r, "distance_km")) else None,
        "u": str(getattr(r, "link_canonical")),
        "id": int(getattr(r, "letter_id")) if hasattr(r, "letter_id") and pd.notnull(getattr(r, "letter_id")) else None,
    })


# -----------------------------
# Folium Karte bauen
# -----------------------------
m = folium.Map(location=CENTER, zoom_start=ZOOM_START, control_scale=True)

# Heatmap als benannte Ebene (damit LayerControl nicht "macro_element_..." anzeigt)
heat_fg = folium.FeatureGroup(name="Heatmap", show=True).add_to(m)
heat_points = [[float(r.lat), float(r.lon), float(r.w)] for r in agg.itertuples(index=False)]
HeatMap(heat_points, radius=18, blur=22, min_opacity=0.15, max_zoom=6).add_to(heat_fg)

# Hotspots (Marker)
cluster = MarkerCluster(name="Hotspots", show=True).add_to(m)

def popup_html_for_place(k: str, place: str, n_total: int, ex_items: list[dict]) -> str:
    ex_li = []
    for it in ex_items:
        who = f"{it.get('s','?')} → {it.get('r','?')}"
        d = it.get("d", "")
        u = it.get("u", "#")
        ex_li.append(f"<li><a href='{u}' target='_blank' rel='noopener'>{who} ({d})</a></li>")

    # Fix: sauberes Quoting -> JS kann sicher laufen
    btn = (
        "<button "
        f"onclick=\"window.rkOpenPlace && window.rkOpenPlace('{k}')\" "
        "style='padding:6px 10px;border:1px solid #ddd;border-radius:8px;background:#fff;cursor:pointer;'>"
        "Alle Korrespondenzen</button>"
    )

    return (
        "<div style='font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;'>"
        f"<div style='font-weight:650;margin-bottom:4px;'>{place}</div>"
        f"<div style='opacity:.8;margin-bottom:8px;'>Briefe am Ort: {n_total}</div>"
        f"<div style='margin: 8px 0;'>{btn}</div>"
        "<div><b>Beispiele:</b></div>"
        "<ul style='padding-left: 18px; margin: 6px 0;'>"
        + ("".join(ex_li) if ex_li else "<li>Keine Beispiele verfügbar.</li>")
        + "</ul></div>"
    )

for r in agg.itertuples(index=False):
    k = coord_key(r.lat, r.lon)
    items = place_index.get(k, [])
    html = popup_html_for_place(k, str(r.place), len(items), items[:EXAMPLES_PER_PLACE])

    folium.Marker(
        location=(float(r.lat), float(r.lon)),
        popup=folium.Popup(html, max_width=420),
        tooltip=f"{r.place} ({len(items)})" if len(items) else str(r.place),
    ).add_to(cluster)

# Linien-FeatureGroup (wird von JS gefüllt)
lines_group = folium.FeatureGroup(name="Linien", show=True).add_to(m)


# -----------------------------
# UI HTML Block (Sidebar + Modal)
# -----------------------------
ui_block = """<style>
  canvas.leaflet-heatmap-layer { pointer-events: none !important; }
  .rk-sidebar { position:absolute; top:10px; left:10px; z-index:9999; background:rgba(255,255,255,0.94);
    padding:10px 12px; border-radius:12px; box-shadow:0 2px 12px rgba(0,0,0,.18); width:360px;
    font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif; }
  .rk-title{ margin:0 0 8px 0; font-size:14px; font-weight:650; }
  .rk-row{ display:flex; gap:8px; align-items:center; margin:6px 0; }
  .rk-row select{ width:100%; padding:6px; border-radius:10px; border:1px solid #ddd; }
  .rk-row button{ padding:6px 10px; border-radius:10px; border:1px solid #ddd; background:#fff; cursor:pointer; }
  .rk-row label{ font-size:12px; display:flex; gap:8px; align-items:center; }
  .rk-meta{ font-size:12px; opacity:.85; margin-top:6px; line-height:1.35; }
  .rk-divider{ height:1px; background:#eee; margin:10px 0; }
  .rk-place h4{ margin:0 0 6px 0; font-size:13px; }
  .rk-list{ max-height:240px; overflow:auto; border:1px solid #eee; border-radius:10px; padding:6px; background:#fff; }
  .rk-list ul{ margin:0; padding-left:18px; }
  .rk-list li{ margin:4px 0; font-size:12px; line-height:1.25; }
  .rk-pager{ display:flex; justify-content:space-between; align-items:center; margin-top:6px; font-size:12px; }
  .rk-muted{ opacity:.75; }
  .rk-chips{ flex-wrap:wrap; }
  .rk-chip{ padding:4px 8px; border-radius:10px; border:1px solid #ddd; background:#fff; cursor:pointer; font-size:12px; }
  .rk-modal-backdrop{ position:absolute; inset:0; z-index:10000; background:rgba(0,0,0,.45); display:none;
    align-items:center; justify-content:center; padding:16px; }
  .rk-modal{ width:min(720px,96vw); background:rgba(255,255,255,0.98); border-radius:16px;
    box-shadow:0 8px 28px rgba(0,0,0,.25); padding:14px 16px; font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif; }
  .rk-modal h3{ margin:4px 0 8px 0; font-size:16px; }
  .rk-modal p{ margin:6px 0; font-size:13px; line-height:1.45; }
</style>

<div class="rk-sidebar">
  <div class="rk-title">RomKorr – Filter & Details</div>
  <div class="rk-row"><label><input type="checkbox" id="rkShowLines" checked /> Linien anzeigen</label></div>
  <div class="rk-row">
    <select id="rkPerson"><option value="">(alle Personen)</option></select>
    <button id="rkReset" title="Filter zurücksetzen">Reset</button>
  </div>
  <div class="rk-row rk-chips" id="rkChips"></div>
  <div class="rk-row">
    <input id="rkYearFrom" type="number" inputmode="numeric" placeholder="Jahr von"
           style="width:100%; padding:6px; border-radius:10px; border:1px solid #ddd;" />
    <input id="rkYearTo" type="number" inputmode="numeric" placeholder="Jahr bis"
           style="width:100%; padding:6px; border-radius:10px; border:1px solid #ddd;" />
  </div>
  <div class="rk-row" style="justify-content: space-between;">
    <button id="rkInfo" title="Info">Info</button>
    <div class="rk-muted" style="font-size:12px;">Tipp: Klick auf einen Hotspot → alle Briefe.</div>
  </div>
  <div class="rk-meta">
    Linien: <span id="rkLineCount">0</span> (von <span id="rkLineTotal">0</span>)<br/>
    Filter matcht Sender <i>oder</i> Empfänger. Zeitraumfilter: Jahreszahl aus dem Datum.
  </div>
  <div class="rk-divider"></div>
  <div class="rk-place" id="rkPlaceBox" style="display:none;">
    <h4 id="rkPlaceTitle">Ort</h4>
    <div class="rk-meta" id="rkPlaceMeta"></div>
    <div class="rk-list"><ul id="rkPlaceList"></ul></div>
    <div class="rk-pager">
      <button id="rkPrev">‹</button>
      <span class="rk-muted" id="rkPageInfo">–</span>
      <button id="rkNext">›</button>
    </div>
    <div class="rk-row" style="justify-content:space-between;">
      <button id="rkClosePlace">Schließen</button>
      <button id="rkDownloadPlace">CSV</button>
    </div>
  </div>
</div>

<div class="rk-modal-backdrop" id="rkModal">
  <div class="rk-modal" role="dialog" aria-modal="true" aria-labelledby="rkModalTitle">
    <h3 id="rkModalTitle">RomKorr – Karte der Korrespondenzen</h3>
    <p>Diese Karte zeigt Korrespondenzen (Briefe) als Hotspots (Orte), Heatmap (Dichte) und Linien (Verbindungen).</p>
    <p><b>Hotspots:</b> Klick auf einen Marker zeigt Beispiele; über „Alle Korrespondenzen“ siehst du alle Briefe für den Ort (mit Paging).</p>
    <p><b>Filter:</b> Wähle eine Person (Sender oder Empfänger). Optional kannst du den Zeitraum über Jahreszahlen einschränken.</p>
    <p class="rk-muted">Hinweis: Viele Linien können Performance kosten. Du kannst Linien jederzeit ausblenden.</p>
    <div class="rk-row" style="justify-content:flex-end;">
      <label style="margin-right:auto; font-size:12px;">
        <input type="checkbox" id="rkDontShowAgain" /> nicht mehr automatisch anzeigen
      </label>
      <button id="rkModalClose">Schließen</button>
    </div>
  </div>
</div>
"""
m.get_root().html.add_child(Element(ui_block))


# -----------------------------
# JS Macro (Linien/Filter/Modal/Sidebar)
# -----------------------------
template = Template(r"""
{% macro html(this, kwargs) %}
<script>
(function() {
  const map = window["{{ this.map_name }}"];
  const LINES = {{ this.lines_json }};
  const PERSONS = {{ this.persons_json }};
  const PLACES = {{ this.places_json }};
  const PAGE_SIZE = {{ this.page_size }};
  const LINES_GROUP = "{{ this.lines_group_name }}";

  function safeText(s) { return (s || "").toString(); }
  function yearFromIso(iso) { const m=(iso||"").toString().match(/^\d{4}/); return m?parseInt(m[0],10):null; }
  function passesYearFilter(iso,yFrom,yTo){ const y=yearFromIso(iso); if(y==null) return true; if(yFrom&&y<yFrom) return false; if(yTo&&y>yTo) return false; return true; }

  function popupHtml(x) {
    const who = `${safeText(x.s)} → ${safeText(x.r)}`;
    const d = safeText(x.d);
    const km = (x.km == null) ? "" : ` · ${Math.round(x.km)} km`;
    const u = safeText(x.u || "#");
    return `<div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;">
      <div style="font-weight:650;margin-bottom:4px;">${who}</div>
      <div style="opacity:.8;margin-bottom:6px;">${d}${km}</div>
      <a href="${u}" target="_blank" rel="noopener">Brief öffnen</a>
    </div>`;
  }

  function downloadCsv(rows, filename) {
    const header = ["id","date","sender","recipient","url"];
    const lines = [header.join(",")].concat(rows.map(r => [
      r.id ?? "",
      (r.d ?? "").toString().replaceAll(",", " "),
      (r.s ?? "").toString().replaceAll(",", " "),
      (r.r ?? "").toString().replaceAll(",", " "),
      (r.u ?? "").toString().replaceAll(",", " "),
    ].join(",")));
    const blob = new Blob([lines.join("\n")], {type:"text/csv;charset=utf-8"});
    const a = document.createElement("a");
    a.href = URL.createObjectURL(blob);
    a.download = filename;
    a.click();
    URL.revokeObjectURL(a.href);
  }

  function drawLines() {
    if (!map || !map.addLayer) return;

    const show = document.getElementById('rkShowLines').checked;
    const filterName = (document.getElementById('rkPerson').value || "").trim();
    const yFrom = parseInt((document.getElementById("rkYearFrom").value || "").trim(), 10) || null;
    const yTo = parseInt((document.getElementById("rkYearTo").value || "").trim(), 10) || null;

    document.getElementById('rkLineTotal').textContent = LINES.length.toString();

    const linesHost = (window[LINES_GROUP] && window[LINES_GROUP].addLayer) ? window[LINES_GROUP] : L.layerGroup().addTo(map);
    linesHost.clearLayers();

    if (!show) { document.getElementById('rkLineCount').textContent="0"; return; }

    let shown=0;
    for (let i=0;i<LINES.length;i++){
      const x=LINES[i];
      if (filterName && x.s!==filterName && x.r!==filterName) continue;
      if (!passesYearFilter(x.iso,yFrom,yTo)) continue;
      const line=L.polyline([x.a,x.b],{weight:2,opacity:0.5});
      line.bindPopup(popupHtml(x),{maxWidth:380});
      line.addTo(linesHost);
      shown++;
    }
    document.getElementById('rkLineCount').textContent=shown.toString();
  }

  function renderPlace(key,page){
    const box=document.getElementById('rkPlaceBox');
    const title=document.getElementById('rkPlaceTitle');
    const meta=document.getElementById('rkPlaceMeta');
    const ul=document.getElementById('rkPlaceList');
    const pageInfo=document.getElementById('rkPageInfo');

    const data=PLACES[key];
    if(!data){ box.style.display="none"; return; }
    window.__rkOpenKey=key;

    const filterName=(document.getElementById('rkPerson').value||"").trim();
    const yFrom=parseInt((document.getElementById('rkYearFrom').value||'').trim(),10)||null;
    const yTo=parseInt((document.getElementById('rkYearTo').value||'').trim(),10)||null;

    const allItems=data.items||[];
    const items=allItems.filter(x=>{
      if(filterName && x.s!==filterName && x.r!==filterName) return false;
      if(!passesYearFilter(x.iso,yFrom,yTo)) return false;
      return true;
    });

    const n=items.length;
    const maxPage=Math.max(0,Math.ceil(n/PAGE_SIZE)-1);
    const p=Math.min(Math.max(0,page),maxPage);
    const start=p*PAGE_SIZE;
    const end=Math.min(n,start+PAGE_SIZE);
    const slice=items.slice(start,end);

    title.textContent=safeText(data.place||"Ort");
    meta.innerHTML=`<span style="opacity:.75">${n} Treffer</span> · Seite ${p+1} / ${maxPage+1}`;

    ul.innerHTML="";
    slice.forEach(x=>{
      const who=`${safeText(x.s)} → ${safeText(x.r)}`;
      const d=safeText(x.d);
      const u=safeText(x.u||"#");
      const li=document.createElement("li");
      li.innerHTML=`<a href="${u}" target="_blank" rel="noopener">${who} (${d})</a>`;
      ul.appendChild(li);
    });

    pageInfo.textContent=(n===0)?"keine Treffer":`${start+1}–${end} von ${n}`;
    box.style.display="block";

    document.getElementById('rkPrev').disabled=(p<=0);
    document.getElementById('rkNext').disabled=(p>=maxPage);
    document.getElementById('rkPrev').onclick=()=>renderPlace(key,p-1);
    document.getElementById('rkNext').onclick=()=>renderPlace(key,p+1);
    document.getElementById('rkDownloadPlace').onclick=()=>downloadCsv(items,`romkorr_${key}.csv`);
  }

  window.rkOpenPlace=function(key){ try{ renderPlace(key,0);}catch(e){console.error(e);} };
  function applyFilters(){ drawLines(); if(window.__rkOpenKey) renderPlace(window.__rkOpenKey,0); }

  function initUi(){
    const sel=document.getElementById('rkPerson');
    PERSONS.forEach(p=>{ const o=document.createElement('option'); o.value=p; o.textContent=p; sel.appendChild(o); });

    const chips=document.getElementById('rkChips');
    chips.innerHTML='';
    PERSONS.slice(0,12).forEach(p=>{
      const b=document.createElement('button'); b.className='rk-chip'; b.type='button'; b.textContent=p;
      b.addEventListener('click',()=>{ sel.value=p; applyFilters(); }); chips.appendChild(b);
    });

    document.getElementById('rkShowLines').addEventListener('change',applyFilters);
    sel.addEventListener('change',applyFilters);
    document.getElementById('rkYearFrom').addEventListener('input',applyFilters);
    document.getElementById('rkYearTo').addEventListener('input',applyFilters);

    document.getElementById('rkReset').addEventListener('click',()=>{
      sel.value=""; document.getElementById('rkYearFrom').value=""; document.getElementById('rkYearTo').value=""; document.getElementById('rkShowLines').checked=true;
      applyFilters();
    });
    document.getElementById('rkClosePlace').addEventListener('click',()=>{
      document.getElementById('rkPlaceBox').style.display='none'; window.__rkOpenKey=null;
    });

    const modal=document.getElementById('rkModal');
    const openBtn=document.getElementById('rkInfo');
    const closeBtn=document.getElementById('rkModalClose');
    const chk=document.getElementById('rkDontShowAgain');
    const open=()=>{ modal.style.display='flex'; };
    const close=()=>{ modal.style.display='none'; if(chk && chk.checked){ try{localStorage.setItem('rk_hide_intro','1');}catch(e){} } };
    openBtn.addEventListener('click',open);
    closeBtn.addEventListener('click',close);
    modal.addEventListener('click',(e)=>{ if(e.target===modal) close(); });
    try{ if(localStorage.getItem('rk_hide_intro')!=='1') open(); }catch(e){ open(); }

    applyFilters();
  }

  if(!map || !map.whenReady) return;
  map.whenReady(initUi);
})();
</script>
{% endmacro %}
""")

macro = MacroElement()
macro._template = template
macro.lines_json = json.dumps(lines_payload, ensure_ascii=False)
macro.persons_json = json.dumps(top_persons, ensure_ascii=False)
macro.places_json = json.dumps(place_payload, ensure_ascii=False)
macro.page_size = PAGE_SIZE_PLACE_LIST
macro.lines_group_name = lines_group.get_name()
macro.map_name = m.get_name()  # CRITICAL: echte Leaflet-map Variable ("map_..."), nicht "figure_..."
m.get_root().add_child(macro)

folium.LayerControl(collapsed=False).add_to(m)

m.save(str(OUT_HTML))
print(f"Wrote: {OUT_HTML.resolve()}")


Wrote: G:\Meine Ablage\CodingProjekte\RomKorr\RomKorr\romkorr_map_fixed.html
